In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm

def plot_2d_normal(mu, cov, min=-10.0, max=10.0, step=0.1):
    x, y = np.mgrid[min:max:step, min:max:step]
    pos = np.dstack((x, y))
    rv = multivariate_normal(mu, cov)
    fig2 = plt.figure()
    ax2 = fig2.add_subplot(111, aspect='equal')
    ax2.contourf(x, y, rv.pdf(pos))
    plt.show()

def plot_1d_normal(mu, variance, min=-0.0, max=25.0, step=0.1):
    x = np.arange(min, max, step)
    rv = norm(mu, np.sqrt(variance))
    plt.plot(x, rv.pdf(x))

In [ ]:
def kf_prediction(mu_t_1, sigma_t_1, u, A, B, Q):
    mu_bar_t = (A @ mu_t_1) + (B @ u)
    sigma_bar_t = (A @ sigma_t_1 @ A.T) + Q
    return mu_bar_t, sigma_bar_t

def kf_correction(mu_bar_t, sigma_bar_t, z, C, R):
    K = sigma_bar_t @ C.T @ np.linalg.inv((C @ sigma_bar_t @ C.T) + R)
    mu = mu_bar_t + (K  @  (z-(C @ mu_bar_t)))
    sigma = (np.identity(sigma_bar_t.shape[0]) - (K @ C)) @ sigma_bar_t
    return mu, sigma

def kf(mu_t_1, sigma_t_1, u, z, A, B, C, Q, R):
    mu_bar_t, sigma_bar_t = kf_prediction(mu_t_1, sigma_t_1, u, A, B, Q)
    mu, sigma = kf_correction(mu_bar_t, sigma_bar_t, z, C, R)
    return mu, sigma, mu_bar_t, sigma_bar_t

In [ ]:
# state is transpose([location, velocity])
mu = np.transpose(np.array([[10, 2]], np.double))
sigma = np.array([[2, 0  ],
                  [0, 0.1]], np.double)

# Each time step, we move according to current velocity
A = np.array([[1, 1],
              [0, 1]], np.double)

# Velocity increases based on how much we increase the throttle
B = np.array([[0],
              [1]], np.double)

# Uncorrelated minor noise in the system
Q = np.array([[1.0, 0  ],
              [0  , 0.1]], np.double)

# GPS-like measurement transform
C = np.array([[1,0]], np.double)

# Noise in measurement
R = np.array([[1]], np.double)

# Acceleration and measurement over time
u = np.array([0,0,0,0,0], np.double)
z = np.array([11, 12, 13, 14, 15], np.double)

# KF
for idx in range(0, len(u)):
    mu_t_1 = mu
    sigma_t_1 = sigma
    mu, sigma, mu_bar_t, sigma_bar_t = kf(
        mu, sigma, np.array([[u[idx]]]), np.array([[z[idx]]]), A, B, C, Q, R)

    plot_1d_normal(mu_t_1.flatten()[0], sigma_t_1.flatten()[0])
    plot_1d_normal(mu_bar_t.flatten()[0], sigma_bar_t.flatten()[0])
    plot_1d_normal(mu.flatten()[0], sigma.flatten()[0])
    plot_2d_normal(mu.flatten(), sigma, min=0.0, max=30.0)